In [1]:
%reset -f
import os
path_input = '.'
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.special import logsumexp
!pip install ./einops-0.8.0-py3-none-any.whl

Processing ./einops-0.8.0-py3-none-any.whl
einops is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [2]:
tab_train = pd.read_csv(f'{path_input}/llm-detect-ai-generated-text/train_essays.csv')
tab_train['set'] = 'train'
#tab_train = tab_train.iloc[:12]
#tab_daigt = pd.read_csv(f'{path_input}/daigt-v2-train-dataset/train_v2_drcat_02.csv')
#tab_daigt.rename(columns = {"label":"generated"}, inplace=True)
#tab_daigt['set'] = 'daigt'
#tab_train = pd.concat([tab_train[['text', 'generated', 'set']],
#                       tab_daigt[['text', 'generated', 'set']]]).reset_index(drop=True)
tab_train['generated'].value_counts()

generated
0    1375
1       3
Name: count, dtype: int64

In [3]:
import torch
import tqdm
device = "cuda:0" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(device, dtype, flush=True)

cuda:0 torch.float16


In [ ]:
from transformers import CodeGenTokenizer, AutoTokenizer, AutoModelForCausalLM
print(device, dtype, flush=True)

dict_llm = {
   #'gpt2' : ('/kaggle/input/save-models/models/gpt2-xl', GPT2LMHeadModel, 1024, dict()),
   #'opt'  : ("/kaggle/input/save-models/models/facebook/opt-2.7b", OPTForCausalLM, 2048, dict()),
   #'bert' : ("/kaggle/input/save-models/models/bert-base-uncased", BertLMHeadModel, 512, {'is_decoder':True}),
   # 'phi3.5' : (f'{path_input}/models/Phi-3.5-mini-instruct', AutoModelForCausalLM, 2048,  dict()),
   # 'Qwen2.5': (f'{path_input}/models/Qwen2.5-3B-Instruct', AutoModelForCausalLM, 2048,  dict()),
   'Qwen2.5': (f'{path_input}/models/Qwen2.5-3B', AutoModelForCausalLM, 2048,  dict()),
}

llm_tokenizer = dict()
llm_model = dict()
for _ in dict_llm:
    llm_tokenizer[_] = AutoTokenizer.from_pretrained(dict_llm[_][0], add_bos_token = True)
    if llm_tokenizer[_].pad_token is None:
        llm_tokenizer[_].pad_token = llm_tokenizer[_].eos_token
    llm_model[_] = dict_llm[_][1].from_pretrained(dict_llm[_][0], torch_dtype=dtype, device_map=device, **dict_llm[_][3])
    print(_, flush=True)

print('done', flush=True)

cuda:0 torch.float16


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

phi3.5
done


In [5]:
def compute_entropy(input_ids, logits, attention_mask, token_type_ids=None):
    with torch.no_grad():
        logits = torch.log_softmax(logits.float(), dim=-1)
        # scores S, W ,P
        tokens = input_ids[:, 1:]
        attention_mask = attention_mask[:, 1:]
        
        entD = torch.sum(logits * torch.exp(logits), dim=-1)[:, 1:]
        entL = torch.gather(logits[:, :-1, :], dim=-1, index = tokens[:,:,None])[:,:,0]
        
        entD = -torch.where(attention_mask!=0, entD, np.nan)
        entL = -torch.where(attention_mask!=0, entL, np.nan)
        
    return entD, entL


def generate_logprob(llm_model, llm_tokenizer, prompt, max_length=None, add_special_tokens = True, padding=False):
    with torch.no_grad():
        device = next(llm_model.parameters()).device
        tokens = llm_tokenizer(prompt, return_tensors="pt",
                               max_length=max_length, truncation=max_length is not None, truncation_strategy = 'longest_first', add_special_tokens=add_special_tokens, padding=padding)
        tokens = {_: tokens[_].to(device) for _ in tokens}
        logits = llm_model(**tokens).logits
        return compute_entropy(logits=logits, **tokens)


class Batch:
    def __init__(self, iterable, size=1):
        self.iterable = iterable
        self.size = size
        self.len = len(range(0, len(self.iterable), self.size))
        
    def __iter__(self):
        l = len(self.iterable)
        n = self.size
        for ndx in range(0, l, n):
            yield self.iterable[ndx:min(ndx + n, l)]
    
    def __len__(self):
        return self.len

In [6]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score

# feats_list = ['Dmed_Qwen2.5', 'Lmed_Qwen2.5', 'Dp05_Qwen2.5', 'Lstd_Qwen2.5', 'meanchr_Qwen2.5',]
feats_list = ['Dmed_phi3.5', 'Lmed_phi3.5', 'Dp05_phi3.5', 'Lstd_phi3.5', 'meanchr_phi3.5',]
classifier = OneClassSVM(verbose=1,  kernel='rbf', gamma='auto',nu=0.05);

list_op = {
    'len':  lambda a, axis: np.sum(np.isfinite(a),axis),
    'med': np.nanmedian,
    'max': np.nanmax,
    'mean': np.nanmean,
    'std': np.nanstd,
    'p05' : lambda a, axis: np.nanpercentile(a, 5,axis=axis),
    'p80' : lambda a, axis: np.nanpercentile(a,80,axis=axis),
    'p90' : lambda a, axis: np.nanpercentile(a,90,axis=axis),
    'p95' : lambda a, axis: np.nanpercentile(a,95,axis=axis),
    'p98' : lambda a, axis: np.nanpercentile(a,98,axis=axis),
    #'lse': logsumexp,
}

In [7]:
batch_size = 4
print(batch_size)
#for _ in dict_llm:
#    print(_, flush=True)
#    texts = [' '.join(['hello',]*dict_llm[_][2]) for i in range(batch_size)]
#    vet = generate_logprob(llm_model[_], llm_tokenizer[_], texts, max_length=dict_llm[_][2], padding=True).cpu().numpy()
#print('done', flush=True)

4


In [8]:
def feature_extraction(tab):
    for index in tqdm.tqdm(tab.index):
        text = tab.loc[index,'text']    
        tab.loc[index,'len_chr'] = len(text)
    for _ in dict_llm:
        print(_, flush=True)
        for index_list in tqdm.tqdm(Batch(tab.index, batch_size)):
            texts = [tab.loc[index,'text'] for index in index_list]
            vetD, vetL = generate_logprob(llm_model[_], llm_tokenizer[_], texts, max_length=dict_llm[_][2], padding=True)
            vetD = vetD.cpu().numpy()
            vetL = vetL.cpu().numpy()
            
            tab.loc[index_list,'meanchr_'+_] = tab.loc[index_list,'len_chr'].values / np.sum(np.isfinite(vetL),-1)
            
            for op in list_op:
                keyD = 'D'+op+'_'+_
                if keyD in feats_list:
                    op_vet = list_op[op](vetD, axis=-1)
                    for index, value in zip(index_list, op_vet):
                        tab.loc[index, keyD] = value
                        
                keyL = 'L'+op+'_'+_
                if keyL in feats_list:
                    op_vet = list_op[op](vetL, axis=-1)
                    for index, value in zip(index_list, op_vet):
                        tab.loc[index, keyL] = value
    return tab

In [9]:
torch.cuda.empty_cache()

In [10]:
if True:
    tab_train = feature_extraction(tab_train)
    print(tab_train.columns)
    print('done training feature extraction', flush=True)

    train_feats = tab_train[tab_train['generated']==0][feats_list].values
    z_mean = np.mean(train_feats, 0, keepdims=True)
    z_std  = np.maximum(np.std(train_feats, 0, keepdims=True), 1e-4)

    classifier.fit((train_feats - z_mean)/z_std)
    
    print('done training', flush=True)
    
else:
    pass

print(z_mean, z_std)

100%|██████████| 1378/1378 [00:00<00:00, 4370.25it/s]

phi3.5



  0%|          | 0/345 [00:00<?, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
You are not running the flash-attention implementation, expect numerical differences.
100%|██████████| 345/345 [02:18<00:00,  2.49it/s]

Index(['id', 'prompt_id', 'text', 'generated', 'set', 'len_chr',
       'meanchr_phi3.5', 'Dmed_phi3.5', 'Lmed_phi3.5', 'Lstd_phi3.5',
       'Dp05_phi3.5'],
      dtype='object')
done training feature extraction
[LibSVM]done training
[[2.24123584 1.61412471 0.07470961 2.69123403 4.35443439]] [[0.30619368 0.32547955 0.06760257 0.18549875 0.22942455]]


*
optimization finished, #iter = 177
obj = 150.050253, rho = 4.968066
nSV = 86, nBSV = 51


In [11]:
tab_test = pd.read_csv(f'{path_input}/llm-detect-ai-generated-text/test_essays.csv')
tab_test = feature_extraction(tab_test)
print('done test feature extraction', flush=True)

100%|██████████| 3/3 [00:00<00:00, 827.28it/s]

phi3.5



100%|██████████| 1/1 [00:00<00:00, 10.29it/s]

done test feature extraction


In [12]:
test_feats = tab_test[feats_list].values
tab_test['generated'] = -1.0*classifier.decision_function((test_feats - z_mean)/z_std)

In [13]:
submission = tab_test[['id','generated']]

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)
pd.read_csv("./submission.csv")

,id,generated
0,0000aaaa,4.967997
1,1111bbbb,4.968066
2,2222cccc,4.968066
